# Create the Features and Labels Files
After calculating the lifted index, we will aggregate all the features and labels together and create one feature file and one label file.

In [1]:
import os
import glob
import pandas as pd

SILVER_GPH20S10K_PATH = '/lakehouse/default/Files/silver/igra2/gph20s10k'
SILVER_LI_PATH = '/lakehouse/default/Files/silver/igra2/li'
GOLD_PARQUET_PATH = '/lakehouse/default/Files/gold/igra2/liftedindex_lr'

# You will need the pyarrow package to create a parquet file
#%conda install pyarrow

In [2]:
# Make sure the destiation folder exists
os.makedirs(GOLD_PARQUET_PATH, exist_ok=True)


In [3]:
def df_from_concatinated_csv(path: str, pattern: str) -> pd.DataFrame:
    """Load multiple csv files into one DataFrame"""
    
    # Get the list of files from a folder
    file_list = glob.glob(path + '/' + pattern)

    # Read each CSV file into DataFrame
    # This creates a list of dataframes
    df_list = (pd.read_csv(file) for file in file_list)

    # Pandas doesn't like to concatinate "empty" DataFrames
    df_list = [dataframe for dataframe in df_list if dataframe.shape[0] > 0]

    # Concatenate all DataFrames
    return pd.concat(df_list, ignore_index=True)

def df_merge(features: pd.DataFrame, labels: pd.DataFrame):
    """Perform inner join on raw features and labels"""

    # Inner Join on id, effectie_date, and hour
    return features \
        .merge(labels, how='inner', on=['id', 'effective_date', 'hour'])

def orchestrate() -> pd.DataFrame:
    """Load and merge the raw files"""
    df_labels = df_from_concatinated_csv(SILVER_LI_PATH, '*-data-li.csv')
    df_features = df_from_concatinated_csv(SILVER_GPH20S10K_PATH, '*-data-gph20s10k.csv')
    return df_merge(df_features, df_labels)


In [4]:
# Get a combined DataFrame of the raw feauters and labels
df = orchestrate()

In [5]:
# Sanity checks
print(f"Row count: {df.shape[0]}")
print("df.info:")
df.info()

Row count: 331801
df.info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331801 entries, 0 to 331800
Columns: 131 entries, id to li
dtypes: float64(128), int64(1), object(2)
memory usage: 331.6+ MB


In [6]:
# View statistics
df.describe()

,hour,day_num,0_gph,0_pres,0_temp,0_dp,0_u,0_v,1_gph,1_pres,...,19_dp,19_u,19_v,20_gph,20_pres,20_temp,20_dp,20_u,20_v,li
count,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000,...,331801.000000,331801.000000,331801.000000,331801.0,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000,331801.000000
mean,6.561436,-0.011635,529.589516,954.507033,13.095635,5.543753,-0.210102,0.596610,799.923079,923.753975,...,-55.353280,20.607472,-0.651793,10000.0,274.551824,-44.422740,-58.393462,21.644289,-0.512284,8.621142
std,6.102106,0.711634,377.485995,42.480249,12.470388,11.377085,2.965843,3.940514,208.630093,23.253499,...,10.204038,16.000494,16.293743,0.0,12.225762,7.088883,9.606863,16.397635,16.603024,10.516321
min,0.000000,-1.000000,198.000000,824.200000,-42.000000,-46.400000,-55.200000,-87.000000,688.100000,824.200000,...,-88.800000,-49.000000,-91.100000,10000.0,231.000000,-68.500000,-90.400000,-46.700000,-105.000000,-27.500000
25%,0.000000,-0.730000,230.000000,919.600000,3.900000,-3.000000,-2.000000,-1.700000,688.100000,918.700000,...,-62.100000,9.400000,-9.500000,10000.0,266.600000,-49.900000,-64.700000,10.100000,-9.700000,-0.100000
50%,12.000000,-0.020000,357.000000,972.800000,14.600000,6.100000,0.000000,0.500000,688.100000,933.100000,...,-55.400000,19.500000,-0.500000,10000.0,276.400000,-44.500000,-58.600000,20.600000,-0.400000,7.300000
75%,12.000000,0.690000,849.000000,987.200000,22.800000,15.200000,1.500000,2.900000,849.000000,938.200000,...,-48.200000,30.900000,8.500000,10000.0,283.500000,-38.800000,-51.700000,32.300000,8.900000,16.200000
max,23.000000,1.000000,1475.000000,1021.100000,43.700000,37.100000,38.600000,46.300000,1475.000000,964.500000,...,-5.000000,109.000000,80.400000,10000.0,520.200000,-4.800000,-6.000000,107.800000,128.600000,57.300000


In [7]:
# Write the parquet file.
df.to_parquet(GOLD_PARQUET_PATH + '/gph20s10k_li.parquet')
